In [354]:
import pandas as pd
import numpy as np

In [355]:
names_path = [
    'MOEXCH_metrics.csv', 
    'MOEXCN_metrics.csv', 
    'MOEXEU_metrics.csv',
    'MOEXFN_metrics.csv',
    'MOEXIT_metrics.csv',
    'MOEXMM_metrics.csv',
    'MOEXOG_metrics.csv',
    'MOEXRE_metrics.csv',
    'MOEXTL_metrics.csv',
    'MOEXTN_metrics.csv'
]

In [356]:
names = []
for i in names_path:
    names.append(i[:6])

In [357]:
results = []
for name in names_path:
    result = pd.read_csv(name, index_col=0).to_numpy()
    results.append(result)

results = np.array(results)

In [358]:
models = pd.read_csv(names_path[0], index_col=0).columns.to_list()
metrics = pd.read_csv(names_path[0], index_col=0).index.to_list()

In [359]:
metrics

['mae', 'mase', 'zbmae', 'smape', 'mda', 'dmda']

In [360]:
results[names.index('MOEXCH'), metrics.index('zbmae'), models.index('mpl')]

1.0287434985180466

In [361]:
result_model = pd.DataFrame(
    index=models
)

for metric in ['mae', 'smape', 'mda', 'dmda']:

    result = pd.Series(
        np.zeros(len(models)),
        index=models,
        name=metric
    )

    for name in names:

        asc = True
        if metric in ('mae', 'smape'):
            asc = False

        ranks = pd.Series(
            results[names.index(name), metrics.index(metric), :],
            index=models,
            name=name
        )

        ranks = ranks.rank(ascending=asc)

        result += ranks
    
    result_model[metric] = result

In [362]:
result_model.to_excel('result_model.xlsx')

In [363]:
result_model.sum(axis=1).sort_values(ascending=False)

ridge       237.5
lr          226.5
xgb         219.0
rf          209.5
mpl         204.0
knn         202.0
gb          195.0
catboost    192.0
adaboost    114.5
dtype: float64

In [364]:
result_model

,mae,smape,mda,dmda
lr,78.0,32.0,64.0,52.5
knn,49.0,78.0,10.0,65.0
ridge,87.0,15.0,76.0,59.5
rf,70.0,24.0,57.0,58.5
gb,51.0,45.0,47.5,51.5
adaboost,18.0,48.0,26.5,22.0
xgb,24.0,79.0,55.0,61.0
catboost,27.0,72.0,57.0,36.0
mpl,46.0,57.0,57.0,44.0


In [365]:

min_zbmaes_for_metrics = pd.DataFrame(index=names)

for metric in metrics:

    min_zbmaes = []
    for name in names:

        min_zbmae = pd.Series(
                        results[names.index(name), metrics.index(metric), :],
                        index=models,
                        name=name
                    )
        if metric in ('mda', 'dmda'):
            min_zbmae = min_zbmae.max()
        
        else:
            min_zbmae = min_zbmae.min()


        min_zbmaes.append(min_zbmae)
    
    min_zbmaes = pd.Series(min_zbmaes, index=names)

    min_zbmaes_for_metrics[metric] = min_zbmaes

In [366]:
min_zbmaes_for_metrics.to_excel('min_zbmaes_for_metrics.xlsx')